# Binary Numbers

Lexer, parser

| Syntactic rules| Semantic Rules |
|:-:|:-:|
| sec -> sec_1 `.` sec_2 | sec_1.v + sec_2.v / 2 ^ sec_2.lon |
| sec -> sec_1 digit | sec_1.v *  2 + dig.v, sec.lon = sec_1.lon + 1 |
| sec -> digit | sec.v = digit.v, sec.long = 1 |
| digit| digit.v = digit.lexval[01]

In [1]:
from sly import Lexer, Parser
import pandas as pd

In [2]:
class BinaryLexer(Lexer):
    literals = {'.'}
    tokens = {'DIGIT'}

    ignore = '\t'

    @_(r'[01]')
    def DIGIT(self, t):
        t.value = int(t.value)
        return t
    
    def error(self, t):
        print('<-'*10, "Illegal character ' {} ' ".format(t.value[0], '->' * 10))
        t.type = "Illegal"
        t.value = t.value[0]
        self.index +=1
        return t

In [3]:
data = pd.read_csv('assets\/binary.csv', dtype=object )
data

,binary
0,01010101
1,001010
2,1
3,0
4,1010101.11


In [4]:
lexer = BinaryLexer()
sentences = pd.read_csv('assets\/binary.csv', dtype=object).binary
pass_or_not = []
all_token_pass = True 

for index, sentence in enumerate(sentences):
    print('-' * 80, "{} Lexically Testing sentence: '{}'".format(index,
          sentence), '-' * 80, sep='\n')
    for token in lexer.tokenize(sentence):
        print(token)
        print(" type = '{}', value = '{}'".format(token.type, token.value))
        if all_token_pass and 'Illegal' in token.type:
            all_token_pass = False
    
    pass_or_not.append('Pass') if all_token_pass else pass_or_not.append('FAIL')
    all_token_pass = True

data['Test'] = pass_or_not


--------------------------------------------------------------------------------
0 Lexically Testing sentence: '01010101'
--------------------------------------------------------------------------------
Token(type='DIGIT', value=0, lineno=1, index=0)
 type = 'DIGIT', value = '0'
Token(type='DIGIT', value=1, lineno=1, index=1)
 type = 'DIGIT', value = '1'
Token(type='DIGIT', value=0, lineno=1, index=2)
 type = 'DIGIT', value = '0'
Token(type='DIGIT', value=1, lineno=1, index=3)
 type = 'DIGIT', value = '1'
Token(type='DIGIT', value=0, lineno=1, index=4)
 type = 'DIGIT', value = '0'
Token(type='DIGIT', value=1, lineno=1, index=5)
 type = 'DIGIT', value = '1'
Token(type='DIGIT', value=0, lineno=1, index=6)
 type = 'DIGIT', value = '0'
Token(type='DIGIT', value=1, lineno=1, index=7)
 type = 'DIGIT', value = '1'
--------------------------------------------------------------------------------
1 Lexically Testing sentence: '001010'
-------------------------------------------------------------

## Parser

In [37]:
class BinaryParser(Parser):
    tokens = BinaryLexer.tokens

    def __init__(self):
        self.names = {}
    
    @_("num")
    def entrada(self, p):
        print("Regla : entrada->num")
        print("El valor es ", p.num)
    
    @_('sec "." sec')
    def num(self, p):
        print("Regla: num-> sec . sec")
        return p.sec0[0] + p.sec1[0] / pow(2, p.sec1[1])
    
    @_('sec dig')
    def sec(self, p):
        print("Regla: sec -> sec dig")
        aux = [p.sec[0] * 2 + p.dig, p.sec[1] + 1]
        print(aux)
        return aux
    
    @_('dig')
    def sec(self, p):
        print("Regla: sec->dig")
        print("Token", p.dig)
        return [p.dig, 1]
    
    @_('DIGIT')
    def dig(self, p):
        print("Regla: dig->DIGIT")
        print("token: ", p.DIGIT)
        return p.DIGIT


In [38]:
sentences = pd.read_csv('assets\/binary.csv', dtype=object)['binary']
lexer = BinaryLexer()
parser = BinaryParser()

# for sentence in sentences:
parser.parse(lexer.tokenize(sentences[1]))


Regla: dig->DIGIT
token:  1
Regla: sec->dig
Token 1
Regla: dig->DIGIT
token:  0
Regla: sec -> sec dig
[2, 2]
Regla: dig->DIGIT
token:  1
Regla: sec -> sec dig
[5, 3]
Regla: dig->DIGIT
token:  0
Regla: sec->dig
Token 0
Regla: num-> sec . sec
Regla : entrada->num
El valor es  5.0
